# Projeto de Reportagem - Violência contra a Mulher

Neste notebook estão os códigos utilizados para as análises e visualização de dados [desta série de reportagens](). Os [dados](https://drive.google.com/drive/folders/1IXTYAIuX85GDC13uwmOPbl5iydAka1nR?usp=sharing) são da Superintendência de Pesquisa e Estratégia de Segurança Pública ([Supesp](https://www.supesp.ce.gov.br/)), uma série histórica, de 2012 a novembro de 2021,  obtida via Lei de Acesso à Informação (LAI).   

## Leitura e limpeza do dataset

In [118]:
import pandas as pd

In [142]:
path = '/content/drive/MyDrive/OPovo/Violência doméstica/baseLAI.csv'
path2 = '/content/drive/MyDrive/OPovo/Violência doméstica/baseExtra2021.csv'

# df1 são os dados obtidos via Lei de Acesso à Informação
df1 = pd.read_csv(path) #Neste arquivo, filtramos os dados até dezembro de 2020! Arquivo no link: https://drive.google.com/file/d/1iidyQWW0TwKQ7DJkPChSv7IOmXU3wWP8/view?usp=sharing
df2 = pd.read_csv(path2)
# df2 são dados de 2021, obtidos no portal XXX
df = pd.concat([df1, df2])
df.head(2)

,AIS,Ano,Data,Hora,Dia da Semana,Gênero da Vítima,Idade da Vítima,Escolaridade da Vítima,Raça da Vítima
0,AIS 04,2012,01/01/2012,0,Domingo,Feminino,37,Ensino Fundamental Incompleto,Não Informada
1,AIS 19,2012,01/01/2012,0,Domingo,Feminino,20,Não Infomada,Não Informada


In [120]:
df['Data'] = pd.to_datetime(df['Data'],format='%d/%m/%Y')
df['Mês'] = df['Data'].dt.month
df['Dia'] = df['Data'].dt.day

In [121]:
df.loc[df['Idade da Vítima'] == 'Não informada','Idade da Vítima'] = 'Não Informada' #Erro de Digitação
df.loc[df['Escolaridade da Vítima'] == 'Não Infomada','Escolaridade da Vítima'] = 'Não Informada' #Erro de Digitação

In [122]:
df['datahora'] = df['Data'].dt.strftime("%Y-%m-%d") +' '+ df['Hora'].astype('string')

## Informações estatísticas básicas do Dataset

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162548 entries, 0 to 17193
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   AIS                     162548 non-null  object        
 1   Ano                     162548 non-null  int64         
 2   Data                    162548 non-null  datetime64[ns]
 3   Hora                    162548 non-null  int64         
 4   Dia da Semana           162548 non-null  object        
 5   Gênero da Vítima        162548 non-null  object        
 6   Idade da Vítima         162548 non-null  object        
 7   Escolaridade da Vítima  162548 non-null  object        
 8   Raça da Vítima          162548 non-null  object        
 9   Mês                     162548 non-null  int64         
 10  Dia                     162548 non-null  int64         
 11  datahora                162548 non-null  string        
dtypes: datetime64[ns](1), int64(4),

In [124]:
df['Ano'].value_counts(normalize = True)

2017    0.143798
2016    0.142807
2019    0.140020
2018    0.132705
2020    0.116292
2021    0.105778
2015    0.082259
2014    0.068355
2013    0.035645
2012    0.032341
Name: Ano, dtype: float64

In [125]:
df['Idade da Vítima'].value_counts()

32     5318
31     5283
33     5280
34     5150
30     5129
       ... 
95        4
96        3
102       2
97        2
98        1
Name: Idade da Vítima, Length: 102, dtype: int64

## Perguntas relevantes contidas nos textos:

**Quantas mulheres foram agredidas por dia, em média?**

In [126]:
df.groupby('Data').size().describe()

count    3622.000000
mean       44.877968
std        22.702755
min         2.000000
25%        26.000000
50%        46.000000
75%        60.000000
max       132.000000
dtype: float64

In [127]:
from scipy import stats
import numpy as np

copia = pd.DataFrame(df.groupby('Data').size().reset_index()[0])
copia[(np.abs(stats.zscore(copia)) < 1).all(axis=1)].describe() #Removendo outliers para um cálculo de média mais consistente: 

,0
count,2358.000000
mean,46.508482
std,12.445508
min,23.000000
25%,36.000000
50%,48.000000
75%,57.000000
max,67.000000


In [128]:
media = copia[(np.abs(stats.zscore(copia)) < 1).all(axis=1)].mean()
print(f'\nA média de casos por hora foi de: {media}')


A média de casos por hora foi de: 0    46.508482
dtype: float64


**Quantas mulheres foram agredidas por hora?**

In [129]:
df.groupby('datahora').size().describe()

count    61257.000000
mean         2.653542
std          1.848400
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max         19.000000
dtype: float64

In [130]:
copia = pd.DataFrame(df.groupby('datahora').size().reset_index()[0])
copia[(np.abs(stats.zscore(copia)) < 1).all(axis=1)].describe()
media = copia[(np.abs(stats.zscore(copia)) < 1).all(axis=1)].mean()

print(f'\nA média de casos por hora foi de: {media}')


A média de casos por hora foi de: 0    2.054023
dtype: float64


**Qual a proporção de casos envolvendo crianças com até 11 anos**

In [131]:
filtro_idade_declarada = df.loc[df['Idade da Vítima'] != 'Não Informada']
filtro_idade_declarada['Idade da Vítima'] = filtro_idade_declarada['Idade da Vítima'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [132]:
total = df.shape[0]
total_idade_declarada = filtro_idade_declarada.shape[0]
total_ate_11anos = filtro_idade_declarada.loc[filtro_idade_declarada['Idade da Vítima'] <= 11].shape[0]
total_ate_11anos

print(f'Total de casos: {total};\nTotal de casos em que a idade foi declarada: {total_idade_declarada};\nTotal de casos com idade declarada até 11 anos: {total_ate_11anos}')

Total de casos: 162548;
Total de casos em que a idade foi declarada: 160642;
Total de casos com idade declarada até 11 anos: 1723


In [133]:
pct_total = total_ate_11anos/total*100
pct_total_idade_declarada = pct_total = total_ate_11anos/total_idade_declarada*100

print(f'Pct(%) de casos de crianças (até 11 anos) do total de registros: {pct_total};\nPct(%) de casos de crianças (até 11 anos) do total de registros com idade declarada:{pct_total_idade_declarada}')

Pct(%) de casos de crianças (até 11 anos) do total de registros: 1.072571307628142;
Pct(%) de casos de crianças (até 11 anos) do total de registros com idade declarada:1.072571307628142


## 1º Ep. [No Ceará, notificação de agressões ignora raça de 73% das mulheres vítimas de violência  doméstica nos últimos 9 anos]()

### Gráfico 1 - Proporção anual de ocorrências de violência doméstica no Ceará sem a identificação de raça das vítimas. 
Visualização no Flourish, [neste link](https://public.flourish.studio/visualisation/8160139/).

In [134]:
# Proporção Anual de especificação de Raça nos registros
copia = df.copy()

copia.loc[copia['Raça da Vítima'] != 'Não Informada','Raça da Vítima'] = 'Informada';

copia = copia.groupby(['Ano','Raça da Vítima']).size().reset_index(name='count')
total_por_ano = copia.groupby('Ano')['count'].transform('sum')
copia['count_normalize'] = round(copia['count'].div(total_por_ano) *100)

copia = copia.pivot(index='Ano',columns='Raça da Vítima', values=['count','count_normalize']).reset_index().astype(int)
copia.columns = ['Ano','Raça Informada','Raça Não Informada','Raça Informada(%)', 'Raça Não Informada(%)']
copia.to_csv('/content/drive/MyDrive/OPovo/Violência doméstica/resumo_declaracao_raca_por_ano.csv')
copia

,Ano,Raça Informada,Raça Não Informada,Raça Informada(%),Raça Não Informada(%)
0,2012,1741,3516,33,67
1,2013,1931,3863,33,67
2,2014,2856,8255,26,74
3,2015,3245,10126,24,76
4,2016,6411,16802,28,72
5,2017,6270,17104,27,73
6,2018,6139,15432,28,72
7,2019,6436,16324,28,72
8,2020,4845,14058,26,74
9,2021,4567,12627,27,73


### Gráfico 2 - Raças que mais sofreram violência / Apenas 27% das ocorrências de violência doméstica no Ceará registra a raça das vítimas
Visualização no Flourish, [neste link](https://public.flourish.studio/visualisation/8160818/).


In [135]:
# Distribuição das ocorrências em que a raça foi declarada:

total = df.loc[df['Raça da Vítima'] != 'Não Informada'].shape[0]

resumo_raca = df.loc[df['Raça da Vítima'] != 'Não Informada']['Raça da Vítima'].value_counts().reset_index()
resumo_raca.columns = ['Raça','Qtd']
resumo_raca['Percentual'] = round(resumo_raca['Qtd']/total *100)
resumo_raca.to_csv('/content/drive/MyDrive/OPovo/Violência doméstica/resumo_raca_declaradas.csv')
resumo_raca

,Raça,Qtd,Percentual
0,Parda,33133,75.0
1,Branca,8772,20.0
2,Preta,2045,5.0
3,Amarela,356,1.0
4,Indígena,135,0.0


### Gráfico 3 - Invisibilidade da raça das vítimas no Ceará / 73% dos registros de violência doméstica no CE não informa a raça das vítimas
Visualização no Flourish, [neste link](https://public.flourish.studio/visualisation/8157749/).

In [136]:
copia = df.copy()
copia.loc[copia['Raça da Vítima'] != 'Não Informada','Raça da Vítima'] = 'Informada';
total = copia.shape[0]
copia = copia.groupby(['Raça da Vítima']).size().reset_index(name='count')
copia['Pct'] = round(copia['count']/total * 100)
copia

,Raça da Vítima,count,Pct
0,Informada,44441,27.0
1,Não Informada,118107,73.0


## Ep2. [Por hora, duas mulheres são vítimas de violência doméstica no CE. Agressões vão de crianças a idosas]()

### Gráfico 4 - Distribuição da idade das vítimas de violência doméstica no Ceará, entre Janeiro de 2012 e Novembro de 2021
Visualização no Flourish, [neste link](https://public.flourish.studio/visualisation/8164729/).


In [137]:
filtro_idade_declarada = df.loc[df['Idade da Vítima'] != 'Não Informada']
filtro_idade_declarada['Idade da Vítima'] = filtro_idade_declarada['Idade da Vítima'].astype(int)

qtd_idade_nao_declarada = df.loc[df['Idade da Vítima'] == 'Não Informada'].shape[0]
print(f'Qtd de ocorrências sem cadastro de idade: {qtd_idade_nao_declarada}')
media = filtro_idade_declarada['Idade da Vítima'].mean()
print(f'Média: {media}')

gIdade_declarada = filtro_idade_declarada.groupby(['Idade da Vítima']).size().reset_index()
gIdade_declarada.columns = ['Idade da Vítima','Qtd']
gIdade_declarada = gIdade_declarada.append({'Idade da Vítima':'Não Informada', 'Qtd': qtd_idade_nao_declarada},ignore_index=True)
gIdade_declarada.to_csv('/content/drive/MyDrive/OPovo/Violência doméstica/distribuicao_idades.csv')
gIdade_declarada

Qtd de ocorrências sem cadastro de idade: 1906
Média: 35.382116756514485


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Idade da Vítima,Qtd
0,0,82
1,1,100
2,2,101
3,3,128
4,4,133
...,...,...
97,97,2
98,98,1
99,100,4
100,102,2


### Gráfico 5 - Distribuição por dia e hora dos registros de violência doméstica no Ceará
Visualização no Flourish, [neste link](https://public.flourish.studio/visualisation/8203486/).

In [138]:
ref_dia = {'Domingo':0,'Segunda':1,'Terça':2,'Quarta':3,'Quinta':4,'Sexta':5,'Sábado':6}

In [139]:
resumo_dia_hora = df.groupby(['Dia da Semana','Hora']).size().reset_index()
resumo_dia_hora.columns = ['Dia','Hora','Qtd']
total = resumo_dia_hora['Qtd'].sum()
resumo_dia_hora['Percentual'] = (resumo_dia_hora['Qtd']/total)*100
resumo_dia_hora['dia_num'] = resumo_dia_hora['Dia'].map(ref_dia)
resumo_dia_hora = resumo_dia_hora.sort_values(['dia_num','Hora'])
resumo_dia_hora.to_csv('/content/drive/MyDrive/OPovo/Violência doméstica/resumo_dia_hora.csv')
resumo_dia_hora

,Dia,Hora,Qtd,Percentual,dia_num
0,Domingo,0,857,0.527229,0
1,Domingo,1,822,0.505697,0
2,Domingo,2,745,0.458326,0
3,Domingo,3,619,0.380811,0
4,Domingo,4,482,0.296528,0
...,...,...,...,...,...
139,Sábado,19,1990,1.224254,6
140,Sábado,20,1882,1.157812,6
141,Sábado,21,1517,0.933263,6
142,Sábado,22,1490,0.916652,6


### Gráfico 6 - Variação acumulada de casos de violência doméstica, divididos por Crianças (0 a 11 anos) e Adolescentes e Adultos (12 anos ou mais), entre 2012 e 2021 
Visualização no Flourish, [neste link](https://public.flourish.studio/visualisation/8194617/).

In [140]:
filtro_idade_declarada = df.loc[df['Idade da Vítima'] != 'Não Informada']
filtro_idade_declarada['Idade da Vítima'] = filtro_idade_declarada['Idade da Vítima'].astype(int)

filtro_idade_declarada.loc[(filtro_idade_declarada['Idade da Vítima'] < 12 ),'gIdade'] = 'Crianças'
filtro_idade_declarada.loc[(filtro_idade_declarada['Idade da Vítima'] >= 12 ),'gIdade'] = 'Adolecentes e Adultos'

resumo_grupo_idade = filtro_idade_declarada.groupby(['Ano','gIdade']).size().reset_index()
resumo_grupo_idade.columns = ['Ano','Grupo Etário','Qtd']
resumo_grupo_idade = resumo_grupo_idade.pivot(index='Ano',columns='Grupo Etário',values ='Qtd')
resumo_grupo_idade['Total'] = resumo_grupo_idade['Crianças'] + resumo_grupo_idade['Adolecentes e Adultos']

resumo_grupo_idade['Pct (%) Crianças'] = round(resumo_grupo_idade['Crianças']/resumo_grupo_idade['Total']*100,1)



VI_criança = resumo_grupo_idade.loc[2012]['Crianças']
VI_adultos = resumo_grupo_idade.loc[2012]['Adolecentes e Adultos']

resumo_grupo_idade['Variação Acumulada - Crianças'] = round((resumo_grupo_idade['Crianças']/VI_criança-1)*100)
resumo_grupo_idade['Variação Acumulada - Adolecentes e Adultos'] = round((resumo_grupo_idade['Adolecentes e Adultos']/VI_adultos-1)*100)
resumo_grupo_idade.to_csv('/content/drive/MyDrive/OPovo/Violência doméstica/resumo_idade_por_grupo.csv')
resumo_grupo_idade

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

Grupo Etário,Adolecentes e Adultos,Crianças,Total,Pct (%) Crianças,Variação Acumulada - Crianças,Variação Acumulada - Adolecentes e Adultos
Ano,,,,,,
2012,5184,33,5217,0.6,0.0,0.0
2013,5701,42,5743,0.7,27.0,10.0
2014,10962,73,11035,0.7,121.0,111.0
2015,13154,103,13257,0.8,212.0,154.0
2016,22724,190,22914,0.8,476.0,338.0
2017,22761,245,23006,1.1,642.0,339.0
2018,20980,275,21255,1.3,733.0,305.0
2019,22196,302,22498,1.3,815.0,328.0
2020,18458,241,18699,1.3,630.0,256.0


### Gráfico 7 - Escolaridade das vítimas / No CE, 40% das vítimas de violência doméstica só tinham sido alfabetizada 

Visualização no Flourish, [neste link](https://public.flourish.studio/visualisation/8164414/).

In [141]:
# Atribuindo os níveis imediatamente inferiores aos registros de formações incompletas
df.loc[df['Escolaridade da Vítima'] == 'Superior Incompleto','Escolaridade da Vítima'] = 'Ensino Médio Completo'
df.loc[df['Escolaridade da Vítima'] == 'Ensino Médio Incompleto','Escolaridade da Vítima'] = 'Ensino Fundamental Completo'
df.loc[df['Escolaridade da Vítima'] == 'Ensino Fundamental Incompleto','Escolaridade da Vítima'] = 'Alfabetizado'

# Removendo o termo "Completo", que não tem mais sentido após o agrupamento
df['Escolaridade da Vítima'] = df['Escolaridade da Vítima'].str.replace(' Completo','')

total = df['Escolaridade da Vítima'].value_counts().sum()
resumo_escolaridade = df['Escolaridade da Vítima'].value_counts().reset_index()
resumo_escolaridade.columns = ['Escolaridade','Qtd']
resumo_escolaridade['Percentual'] = round(resumo_escolaridade['Qtd']/total *100,2)

resumo_escolaridade.to_csv('/content/drive/MyDrive/OPovo/Violência doméstica/Resumo_escolaridade.csv')
resumo_escolaridade

,Escolaridade,Qtd,Percentual
0,Alfabetizado,65104,40.05
1,Ensino Médio,41302,25.41
2,Ensino Fundamental,34057,20.95
3,Não Informada,8327,5.12
4,Superior,7448,4.58
5,Não Alfabetizado,6310,3.88
